# Actividad 3: Entrenamiento de un modelo de Machine Learning (ML)

El proceso de entrenamiento de un modelo de ML consiste en proporcionar datos de entrenamiento, de los cuales aprende un algoritmo de ML (es decir, el algoritmo de aprendizaje). El término modelo de ML se refiere al artefacto de modelo que se crea en el proceso de entrenamiento. Los datos de entrenamiento se obtienen a través del proceso de ETL generado, el cual, debe contener la variable objetivo o tartet y cada de una de las variables que ayudan al modelo a encontrar la respuesta/salida correcta. Una vez los datos son obtenidos, inicial el proceso de entrenamiento. En un proceso de entrenamiento podemos encontrar diferentes pasos que varian dependiendo de la complejidad del problema, que son: preprocesamiento de datos, definir arquitectura del modelo, entrenar y evaluar el desempeño.

En la presente actividad entrenaremos un modelo de ML para identificar si un tipo de cáncer de mama puede ser **Maligno** o **Benigno**, la cual, vendría a ser nuestra variable objetivo o target. En el dataset, haremos uso de las siguientes características.

* radius: media de las distancias del centro a los puntos del perímetro
* texture: desviación estándar de los valores de la escala de grises
* perimeter
* area
* smoothness: variación local en longitudes de radio
* compactness:(perimeter^2 / area - 1.0)
* concavity: gravedad de las porciones cóncavas del contorno
* concave points: número de porciones cóncavas del contorno
* symmetry
* fractal dimension: (aproximación a la línea de costa" - 1)



# Imports 

Librerias que se necesitan para el desarrollo de la actividad

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Configurando pandas para visualización de 100 columnas

In [2]:
pd.options.display.max_columns = 100

# Lectura del dataset

In [3]:
path = 'https://github.com/jjguajo/activities_ai/blob/main/data_activity_3_ml.csv'
df = pd.read_csv(path, on_bad_lines='skip')

In [4]:
data = pd.read_csv("data.csv")

In [5]:
# Dimensión del dataset
data.shape

(569, 33)

In [6]:
# Primeros 5 registros del dataset
data.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
# Inspeccionando la variable objetivo
data.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [8]:
# Validando si hay presentes valores NULLs en el dataset
data.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

_**Generalmente, los IDs que identifican registros en un dataset no aportan información significativa al entrenamiento de un modelo de ML. Al igual que los valores NULL**_

In [9]:
data.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

# Entrenamiento del modelo

_**Como se mencionó al inicio del notebook. Para esta actividad, solo utilizaremos algunas features del dataset completo**_

In [10]:
features_finales = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", 
    "fractal_dimension_mean", "diagnosis"
]

In [11]:
# Seleccionamos solo las features de interés del dataset completo
dataset_final = data[features_finales]

In [12]:
# Ver tipos de datos de nuestro conjunto de datos
dataset_final.dtypes

radius_mean               float64
texture_mean              float64
perimeter_mean            float64
area_mean                 float64
smoothness_mean           float64
compactness_mean          float64
concavity_mean            float64
concave points_mean       float64
symmetry_mean             float64
fractal_dimension_mean    float64
diagnosis                  object
dtype: object

Se puede observar que todas las features son de tipo númerico. En un proceso de entrenamiento de un modelo de ML nos podemos encontrar con otros tipos de datos, como los categóricos. Para efectos de creación del proceso de entrenamiento que nos ofrece Sklearn, crearemos una variable categórica. **Nota:** la menera de crear la variable categórica se realiza de aleatoria, sin embargo, en ciencia de datos existe un proceso llamado **Feature Engineering**, en donde se detallan otra maneras de realizrlo

In [13]:
# Variable categórica creada a partir de la feature fractal_dimension_mean. 
dataset_final["calc_fractal_dimension_mean"] = np.where(
    1, dataset_final["fractal_dimension_mean"] < 0.055, 
    0
)

<ipython-input-13-2ddb9cafa4af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final["calc_fractal_dimension_mean"] = np.where(


In [14]:
# Eliminamos la variable númerica que usamos para crear la variable categórica
dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)

<ipython-input-14-b89a3607ea7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)


## Pipeline de Sklearn para el entrenamiento de un modelo de ML

[Sklearn](https://scikit-learn.org/stable/) ofrece una serie de pasos en los cuales podemos definir la arquitectura de nuestro modelo. En estos pasos, podemos definir el preprocesamiento de nuestras variables númericas y categóricas, realizar un feature unión y especificar el algoritmo de ML a usar


In [15]:
# Clase que realiza el fit y transform a una variable
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

### Definicíon de pipeline variables númericas

In [16]:
# Features numéricas
numerical = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean"
]

In [17]:
# ColumnSelector: selecciona las columnas númericas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con la mediana de todos los valores de la feature
num_pipe = Pipeline([
    ('selector', ColumnSelector(numerical)),
    ('imputer', SimpleImputer(strategy='median'))
])

# Definicíon de pipeline variables categóricas

In [18]:
# Features categóricas
categorical = ["calc_fractal_dimension_mean"]

In [19]:
# ColumnSelector: selecciona las columnas categóricas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con un valor fijo
# OneHotEncoder: codificación de variables categóricas
cat_pipe = Pipeline([
    ('selector', ColumnSelector(categorical)),
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [20]:
# Feature union de todas las features categóricas y numéricas
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

In [21]:
# Pipeline final del proceso de entrenamiento
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svm.SVC())
])

In [22]:
# Se elimina la variable target de los datos que contienen las features 
X = dataset_final.drop(['diagnosis'], axis = 1)

In [23]:
# Se genera en una variable separada el target del problema, en este caso "Diagnosis"
y = dataset_final['diagnosis']

In [24]:
# De todo el conjunto de datos se toman crean datos de train y test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
print("Size of training set:", X_train.shape)
print("Size of test set:", X_test.shape)

Size of training set: (398, 10)
Size of test set: (171, 10)


In [25]:
# Entrenamiento
model = pipe.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
# Predicción sobre los datos de test
y_pred = model.predict(X_test)

# Validación performance del modelo

Evaluamos el desempeño del modelo haciendo uso de la librería [Classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)  que otorga Sklearn. En esta tabla podemos encontrar diferentes métricas, en donde cada una se utiliza dependiendo del problema a resolver y como afecta una mala predicción por parte del modelo. Para esta actividad, vamos a usar el **Accuracy**, la cual, lo que nos muestra es el porcentaje de casos que el modelo ha acertado, en esta actividad, obtenemos un accuracy del **0.88**

In [27]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.85      0.98      0.91       108
           M       0.96      0.70      0.81        63

    accuracy                           0.88       171
   macro avg       0.90      0.84      0.86       171
weighted avg       0.89      0.88      0.87       171



# Ajuste de Hiperparámetros del modelo

Los hiperparámetros se definen como los parámetros definidos explícitamente por el usuario para controlar el proceso de aprendizaje. Cada algortimo de aprendizaje automático contiene hiperparámetros específicos que se seleccionan y optimizan durante el proceso de entrenamiento con el objetivo de mejorar el desempeño del modelo.

Para la actividad, utilizaremos ***GridSearchCV***, el cual, es una librería se [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) que realiza una búsqueda exhaustiva sobre valores de parámetros específicos para un estimador, en este caso, para el [Support Vector Machine (SVM)](https://scikit-learn.org/stable/modules/svm.html)


In [28]:
# Definimos un rango de búsqueda de mejores hiperparámetros
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

In [29]:
# Búsqueda de mejores hiperaparámetros. Esta busqueda se realiza utilizando los datos de entrenamiento
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}])

In [30]:
# Mejores hiperparametros encontrados por GridSearch
clf.best_params_

{'C': 10, 'kernel': 'linear'}

In [31]:
# Generar el pipeline con los mejores hiperparametros
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svm.SVC(C=10, kernel='linear'))
])

In [32]:
# Entrenar nuevamente
model = pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [33]:
# Predecir nuevamente sobre los datos de test
y_pred = model.predict(X_test)

In [34]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.94      0.91      0.92       108
           M       0.85      0.90      0.88        63

    accuracy                           0.91       171
   macro avg       0.90      0.91      0.90       171
weighted avg       0.91      0.91      0.91       171



Podemos observar que el accuracy aumenta del 0.88 al 0.91. Con esto, podemos notar que la optimización de los hiperparámetros del modelo es un paso que ayuda al modelo a tener un mejor desempeño en diferentes tipos de tareas, para esta actividad, una tarea centrada en la clasificación de dos tipos de cancer de mama